In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn import linear_model
import pickle

### LASSO

répartition des données

In [3]:
df = pd.read_csv('house_data_cleaned.csv')
y = df['price']
X = df[[
#     'id',
    # 'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',    
    # 'sqft_basement',
    'yr_built',
    'yr_renovated',
    'zipcode', #
    'lat',
    'long',
#     'sqft_living15',
#     'sqft_lot15',
    # 'yr_best',
    # 'yr_sold',
    'yr_gap',
    # 'comparison',
    ]]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=42
    )

### Preparation du pipeline

Gestion des données numériques

In [5]:
numeric_features = [
    # 'date',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'yr_built',
    'yr_renovated',
    'floors',
    'sqft_above',
    # 'sqft_basement',
    'lat',
    'long',
    # 'yr_best',
    # 'yr_sold',
    'yr_gap',
    # 'comparison'
    ]

#on genere les 3 scalers
numeric_transformer_minmax = Pipeline([
    ('minmax', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=3))
    ])

numeric_transformer_std = Pipeline([
    ('standard', StandardScaler()),
    ('poly', PolynomialFeatures(degree=3))
    ])

numeric_transformer_rbst = Pipeline([
    ('standard', RobustScaler()),
    ('poly', PolynomialFeatures(degree=3))
    ])

Gestion des données categorielles

In [6]:
categorial_features = [
    'view',
    'condition',
    'grade',
    'zipcode',
    ]

categorial_transformer = OneHotEncoder(sparse_output=True)

In [7]:
preprocessor_minmax = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_minmax, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

preprocessor_std = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_std, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

preprocessor_rbst = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_rbst, numeric_features),
        ('cat', categorial_transformer, categorial_features)
    ],
    remainder='passthrough'
)

Estimateur

In [12]:
lasso = linear_model.Lasso(alpha=0.05)

In [13]:
pipe = Pipeline([
    ('prep', preprocessor_minmax),
    ('lasso', lasso)
])

trained_pipe = pipe.fit(X_train, y_train)
trained_pipe.predict(X_test)
trained_pipe.score(X_test, y_test)



/home/apprenant/miniconda3/envs/machine_learning/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.336e+14, tolerance: 2.310e+11
  model = cd_fast.enet_coordinate_descent(


0.8640105741227155

enregistrer le modèle d'entrainement choisi sur pickle

In [ ]:
pickle.dump(trained_pipe, open('ppdjangoapp/trained_pipe.pkl', 'wb'))

historique des scores

In [10]:
scores = {
    # alpha = 0.1
    'minmax_lasso' : 0.8387818153900362,
    'std_lasso' : 0.8387816477252281,
    'rbst_lasso' : 0.8387816518697903,
    # alpha = 0.1
    'minmax_lasso_poly2' : 0.8471566129591712,
    'std_lasso_poly2' : 0.848558099929865,
    'rbst_lasso_poly2' : 0.8483787840075337,
    # alpha = 0.05 mais ne converge pas
    'minmax_lasso_poly3_05' : 0.8640105741227155,
    # alpha = 0.1
    'minmax_lasso_poly3' : 0.8630780852596053,
    'std_lasso_poly3' : 0.8630780852596053,
    'rbst_lasso_poly3' : 0.8616635749230119,
}

bestscore = max(scores.values())
bestscore = [k for (k, val) in scores.items() if val == bestscore]
print(bestscore[0])

minmax_lasso_poly3_05


essai du modele entrainé

In [11]:
import random as rd
#on genere un chiffre aléatoire compris en 0 et la longueur de la liste X_test
randomint = rd.randint(0,len(X_test)+1)
predict = round(trained_pipe.predict(X_test.iloc[[randomint]])[0])
real = round(y.iloc[[randomint]].values[0])
difference = round(abs(real - predict))
accuracy = str(round((((real-difference) * 100) / real), 2)) + "%"
print(f"Valeur prédite : {predict} \n Valeur réelle : {real} \n Différence : {difference}, \n Précision : {accuracy}")

Valeur prédite : 524367 
 Valeur réelle : 425000 
 Différence : 99367, 
 Précision : 76.62%
